# Analysis of BMC solving with Z3

Pandas cheatsheet: https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

There are 4 different encoding parameters:

  - **phi** whether the VC is encoded with phi-assignments. This is traditional encoding for `SeaHorn`
  - **gsa** whether the program is converted to Gated SSA before VCGen. In this case, there are no phi-assignments
  - **ite** whether phi-nodes are encoded with ite-expressions
  - **noite** whether phi-nodes are encoded with disjunction and equalities (traditional for `SeaHorn`)
  
Four combinations are considered: {ite, noite} x {phi, gsa}. 
The combination (ite, phi) is traditional for `SeaHorn`. The combination (ite, gsa) is meaningles since under gsa, there are no phi-nodes, and ite has no effect. Yet, the result show some difference between (ite, gsa) and (noite, gsa). This needs to be investigated further. Possibly the difference is due to usual SAT-solver non-determinism.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
df = pd.read_csv('bmc-z3.csv')
# move timeout and unknown to maximal time
df['time'].fillna(value=900, inplace=True)
# show a snapshot of data for visual validation
df.head()

In [ ]:
# quick overview of solved/unsolved instances
df[['status','index']].groupby(['status']).count()

In [ ]:
# find all unknown results. Usually means that something went wrong, not just timeout
df.query('status == "unknown"')

In [ ]:
# split results into 4 groups
def flt_index(df, s):
    return df[df['index'].str.contains(s)]

noite_gsa = flt_index(df, r'\.noite.gsa')
ite_gsa = flt_index(df, r'\.ite.gsa')
ite_phi = flt_index(df, r'\.ite.phi')
noite_phi = flt_index(df, r'\.noite.phi')

## Solved / Unsolved for each group

In [ ]:
noite_gsa.groupby('status').count()

In [ ]:
ite_gsa.groupby('status').count()

In [ ]:
ite_phi.groupby('status').count()

In [ ]:
noite_phi.groupby('status').count()

In [ ]:
# create a file column to join different result sets
def index_to_file(df):
    return df.assign(file=df['index'].str.extract(r'([^.]+)'), expand=False)
noite_phi = index_to_file(noite_phi)
ite_phi = index_to_file(ite_phi)
noite_gsa = index_to_file(noite_gsa)
ite_gsa = index_to_file(ite_gsa)

In [ ]:
# join on file
phi = ite_phi.merge(noite_phi, on='file', suffixes=('_ite', '_noite'))

In [ ]:
# solved / unsolved based on ite parameter
phi.groupby(['status_ite', 'status_noite']).count()[['file']]

In [ ]:
# merge gsa sets and show solved / unsolved
gsa = ite_gsa.merge(noite_gsa, on='file', suffixes=('_ite', '_noite'))
gsa.groupby(['status_ite', 'status_noite']).count()[['file']]

In [ ]:
# set default figure size for plots to be 7 by 7 inches
plt.rcParams["figure.figsize"] = (7, 7)
# create a new figure 20 by 10 inches wide
fig = plt.figure(figsize=(20,10))
# break figure into two sub-plots
# ax1: the number '121' is parsed as plot 1 of 1x2 grid
ax1 = plt.subplot(121)
# ax2: the number '122' is parsed as plot 2 of 1x2 grid
ax2 = plt.subplot(122)
# plot a diagonal line to make scatter plots easier to read
x = np.linspace(-5, 950, 2)
ax1.plot(x, x, color='red')
ax2.plot(x, x, color='red')
# plot succeeded instances
gsa.query('status_ite=="sat" or status_ite=="unsat"').plot.scatter(x='time_ite', y='time_noite', ax=ax1, title='GSA')
phi.query('status_noite=="sat" or status_ite=="unsat"').plot.scatter(x='time_ite', y='time_noite', ax=ax2, title='PHI')

In [ ]:
# a single plot is easier to create
ax = phi.query('status_noite=="sat"').plot.scatter(x='time_ite', y='time_noite')
ax.plot(x, x, color='red')

In [ ]:
# summary of the plot. Useful to validate what is visible from the picture
phi.groupby(['status_ite', 'status_noite'])[['time_ite', 'time_noite']].describe()

In [ ]:
import sys
# install latest version of seaborn
!{sys.executable} -m pip install seaborn=0.9.0
# seaborn provides useful fancy plots
import seaborn as sns
sns.set(style='ticks', palette='Set2')

In [ ]:
# use additional semantic features to separate sat/unsat instances
fig = plt.figure(figsize=(10,10))
g = sns.scatterplot(x="time_ite", y="time_noite", hue="status_noite", style='status_ite', data=phi);
sns.despine()
g.plot(x, x)

In [ ]:
!{sys.executable} -m pip install plotly
import plotly.express as px
# interactive graph for data exploration
fig = px.scatter(phi, x="time_ite", y="time_noite", color="status_noite", symbol="status_ite")
fig.show()